In [1]:
from collections import defaultdict
from pathlib import Path

from bs4 import BeautifulSoup
from requests import Session
import pandas as pd

In [2]:
results = 'results'
fixtures = 'fixtures'

In [3]:
url = 'https://www.bbc.com/sport/american-football/'

In [4]:
session = Session()

headers = headers = {'User-Agent': ('Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) '
                          'AppleWebKit/537.36 (KHTML, like Gecko) '
                          'Chrome/39.0.2171.95 Safari/537.36'),
                    'Content-Type': 'application/json'}

session.headers.update(headers)

In [5]:
fixtures_url = f'{url}{fixtures}'

r = session.get(fixtures_url)

fixtures_soup = BeautifulSoup(r.content,'html.parser')

games_soup = fixtures_soup.find('div', {'id':'sp-c-filter-contents'})

games = games_soup.find_all('span',{'class':'qa-fixture-block'})

In [6]:
data = defaultdict(list)
for game in games:
    data['game_date'].append(game.h3.get_text())
    data['game_round'].append(game.h5.get_text())
    data['games'].append([{'home':home['title'],'away':away['title']} for home, away in [g.find_all('abbr') for g in game.find_all('li')]])
    data['time'].append([{'time':time.get_text()} for time in game.find_all('span', class_='sp-c-fixture__number--time')])
    data['venue'].append([{'venue':venue.get_text()} for venue in game.find_all('span', class_='sp-c-fixture__venue')])

In [7]:
df = pd.DataFrame(data)

In [8]:
df.head()

,game_date,game_round,games,time,venue
0,Friday 6th September 2019,Round 1,"[{'home': 'Chicago Bears', 'away': 'Green Bay ...",[{'time': '00:20'}],[{'venue': 'Venue: Soldier Field'}]
1,Sunday 8th September 2019,Round 1,"[{'home': 'Carolina Panthers', 'away': 'Los An...","[{'time': '17:00'}, {'time': '17:00'}, {'time'...","[{'venue': 'Venue: Bank of America Stadium'}, ..."
2,Monday 9th September 2019,Round 1,"[{'home': 'New England Patriots', 'away': 'Pit...",[{'time': '00:20'}],[{'venue': 'Venue: Gillette Stadium'}]
3,Tuesday 10th September 2019,Round 1,"[{'home': 'New Orleans Saints', 'away': 'Houst...","[{'time': '23:10'}, {'time': '02:20'}]","[{'venue': 'Venue: Mercedes-Benz Superdome'}, ..."
4,Friday 13th September 2019,Round 2,"[{'home': 'Carolina Panthers', 'away': 'Tampa ...",[{'time': '00:20'}],[{'venue': 'Venue: Bank of America Stadium'}]


In [9]:
games = df.explode('games')
time = df.explode('time')
venue = df.explode('venue')

In [10]:
games.drop(columns=['time','venue'], inplace=True)
time.drop(columns=['games','venue'], inplace=True)
venue.drop(columns=['time','games'], inplace=True)

In [11]:
games['time'] = time['time']
games['venue'] = venue['venue']

In [12]:
df = games

In [36]:
df.head()

,game_date,game_round,games,time,venue
0,Friday 6th September 2019,Round 1,"{'home': 'Chicago Bears', 'away': 'Green Bay P...",{'time': '00:20'},{'venue': 'Venue: Soldier Field'}
1,Sunday 8th September 2019,Round 1,"{'home': 'Carolina Panthers', 'away': 'Los Ang...",{'time': '17:00'},{'venue': 'Venue: Bank of America Stadium'}
1,Sunday 8th September 2019,Round 1,"{'home': 'New York Jets', 'away': 'Buffalo Bil...",{'time': '17:00'},{'venue': 'Venue: MetLife Stadium'}
1,Sunday 8th September 2019,Round 1,"{'home': 'Minnesota Vikings', 'away': 'Atlanta...",{'time': '17:00'},{'venue': 'Venue: US Bank Stadium'}
1,Sunday 8th September 2019,Round 1,"{'home': 'Miami Dolphins', 'away': 'Baltimore ...",{'time': '17:00'},{'venue': 'Venue: Hard Rock Stadium'}


In [50]:
df['time'] = df['time'].apply(lambda x: x['time'])

In [75]:
df['game_dt'] = df.apply(lambda x: f'{x["game_date"]} {x["time"]}', axis=1)

In [76]:
df.head()

,game_date,game_round,games,time,venue,game_dt
0,Friday 6th September 2019,Round 1,"{'home': 'Chicago Bears', 'away': 'Green Bay P...",00:20,{'venue': 'Venue: Soldier Field'},Friday 6th September 2019 00:20
1,Sunday 8th September 2019,Round 1,"{'home': 'Carolina Panthers', 'away': 'Los Ang...",17:00,{'venue': 'Venue: Bank of America Stadium'},Sunday 8th September 2019 17:00
2,Sunday 8th September 2019,Round 1,"{'home': 'New York Jets', 'away': 'Buffalo Bil...",17:00,{'venue': 'Venue: MetLife Stadium'},Sunday 8th September 2019 17:00
3,Sunday 8th September 2019,Round 1,"{'home': 'Minnesota Vikings', 'away': 'Atlanta...",17:00,{'venue': 'Venue: US Bank Stadium'},Sunday 8th September 2019 17:00
4,Sunday 8th September 2019,Round 1,"{'home': 'Miami Dolphins', 'away': 'Baltimore ...",17:00,{'venue': 'Venue: Hard Rock Stadium'},Sunday 8th September 2019 17:00


In [77]:
 df.to_json('data/fixtures.json', orient='records')

In [78]:
dt = pd.read_json('data/fixtures.json', orient='records')

In [79]:
dt.dtypes

game_date     object
game_round    object
games         object
time          object
venue         object
game_dt       object
dtype: object

In [80]:
dt.head()

,game_date,game_round,games,time,venue,game_dt
0,Friday 6th September 2019,Round 1,"{'home': 'Chicago Bears', 'away': 'Green Bay P...",00:20,{'venue': 'Venue: Soldier Field'},Friday 6th September 2019 00:20
1,Sunday 8th September 2019,Round 1,"{'home': 'Carolina Panthers', 'away': 'Los Ang...",17:00,{'venue': 'Venue: Bank of America Stadium'},Sunday 8th September 2019 17:00
2,Sunday 8th September 2019,Round 1,"{'home': 'New York Jets', 'away': 'Buffalo Bil...",17:00,{'venue': 'Venue: MetLife Stadium'},Sunday 8th September 2019 17:00
3,Sunday 8th September 2019,Round 1,"{'home': 'Minnesota Vikings', 'away': 'Atlanta...",17:00,{'venue': 'Venue: US Bank Stadium'},Sunday 8th September 2019 17:00
4,Sunday 8th September 2019,Round 1,"{'home': 'Miami Dolphins', 'away': 'Baltimore ...",17:00,{'venue': 'Venue: Hard Rock Stadium'},Sunday 8th September 2019 17:00


In [81]:
dt['game_dt'].apply(pd.to_datetime)

0     2019-09-06 00:20:00
1     2019-09-08 17:00:00
2     2019-09-08 17:00:00
3     2019-09-08 17:00:00
4     2019-09-08 17:00:00
              ...        
251   2019-12-29 18:00:00
252   2019-12-29 18:00:00
253   2019-12-29 21:25:00
254   2019-12-29 21:25:00
255   2019-12-29 21:25:00
Name: game_dt, Length: 256, dtype: datetime64[ns]

In [ ]:
# Logos
name of the team.lower().replace(' ','-')
team =los-angeles.rams
# f'http://loodibee.com/wp-content/uploads/nfl-{team}-team-logo-2-300x300.png'

In [11]:
logos_url = 'http://loodibee.com/nfl/'

In [9]:
folder = './static/logos/'
Path(folder).mkdir(parents=True, exist_ok=True)

In [12]:
s = session.get(logos_url)

In [13]:
logos_soup = BeautifulSoup(s.content,'html.parser').find_all('figure')

In [14]:
team_name_logo = [(logo.a['href'][:-1],logo.noscript.img['src']) for logo in logos_soup]
team_name_logo = team_name_logo[:-3]

In [15]:
for team, image_url in team_name_logo:
    response =session.get(str(image_url), stream=True)
    if response.ok:
        print(f'Success Team: {team.replace("-"," ").title()}')
        with open(f'{folder}{team}.png', 'wb') as f:
            f.write(response.content)
    else:
        print(f'Failed Team: {team.replace("-"," ").title()}')

Success Team: Arizona Cardinals
Success Team: Atlanta Falcons
Success Team: Baltimore Ravens
Success Team: Buffalo Bills
Success Team: Carolina Panthers
Success Team: Chicago Bears
Success Team: Cincinnati Bengals
Success Team: Cleveland Browns
Success Team: Dallas Cowboys
Success Team: Denver Broncos
Success Team: Detroit Lions
Success Team: Green Bay Packers
Success Team: Houston Texans
Success Team: Indianapolis Colts
Success Team: Jacksonville Jaguars
Success Team: Kansas City Chiefs
Success Team: Los Angeles Chargers
Success Team: Los Angeles Rams
Success Team: Miami Dolphins
Success Team: Minnesota Vikings
Success Team: New England Patriots
Success Team: New Orleans Saints
Success Team: New York Giants
Success Team: New York Jets
Success Team: Oakland Raiders
Success Team: Philadelphia Eagles
Success Team: Pittsburgh Steelers
Success Team: San Francisco 49Ers
Success Team: Seattle Seahawks
Success Team: Tampa Bay Buccaneers
Success Team: Tennessee Titans
Success Team: Washington 